In [ ]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

from dotenv import load_dotenv
import os

import pandas as pd

from collections import defaultdict

# ---<---<--<--@    hasta aquí nuestras librerías   @-->-->--->---

# Identifica nuestras claves guardadas con dotenv
load_dotenv() 
client_id = os.getenv("CLIENT_ID")
client_secret = os.getenv("CLIENT_SECRET")

# Autenticación API Spotify
auth_manager = SpotifyClientCredentials(client_id, client_secret)
sp = spotipy.Spotify(auth_manager=auth_manager)

# Función para automatizar la extracción de lo que queremos solicitar:
# -> artista, género, canción/album, año de lanzamiento, id_lanzamiento

def extraer_datos_por_genero_y_rango(genero, rango_anyo, resultados_por_anyo=200):
    albumes_por_anyo = defaultdict(list)
    canciones_por_anyo = defaultdict(list)

    for o in range(0, 200, 50):
        artistas = sp.search(q=f'genre:{genero}', type='artist', limit=50, offset=o)
        for artista in artistas['artists']['items']:
            artista_id = artista['id']
            artista_nombre = artista['name']

            # aquí extraemos álbumes
            albumes_data = sp.artist_albums(artista_id, album_type=['album'], limit=50)
            for album in albumes_data['items']:
                try:
                    album_lanzamiento = int(album['release_date'][:4])
                    if album_lanzamiento in rango_anyo and len(albumes_por_anyo[album_lanzamiento]) < resultados_por_anyo:
                        albumes_por_anyo[album_lanzamiento].append({
                            'artista': artista_nombre,
                            'genero': genero,
                            'tipo': album['album_type'],
                            'nombre': album['name'],
                            'año': album_lanzamiento,
                            'id': album['id']
                        })
                except (KeyError, ValueError, TypeError):
                    continue

            # aquí extraemos canciones
            try:
                top_tracks = sp.artist_top_tracks(artista_id)
                for track in top_tracks['tracks']:
                    try:
                        track_lanzamiento = int(track['album']['release_date'][:4])
                        if track_lanzamiento in rango_anyo and len(canciones_por_anyo[track_lanzamiento]) < resultados_por_anyo:
                            canciones_por_anyo[track_lanzamiento].append({
                                'artista': artista_nombre,
                                'genero': genero,
                                'tipo': 'cancion',
                                'nombre': track['name'],
                                'año': track_lanzamiento,
                                'id': track['id']
                            })
                    except (KeyError, ValueError, TypeError):
                        continue
            except 'SpotifyException':
                continue

    # devuelve un diccionario con dos listas separadas
    return {
        'albumes': [{'tipo': 'album', **data} for album_lanzamiento in rango_anyo for data in albumes_por_anyo[album_lanzamiento][:resultados_por_anyo]],
        'canciones': [{'tipo': 'cancion', **data} for anyo in rango_anyo for data in canciones_por_anyo[anyo][:resultados_por_anyo]]
    }



In [ ]:
### A PARTIR DE AQUÍ EL USO, INCLUYE LA EXTRACCIÓN A CSV

tastyinfo_2018_2022 = extraer_datos_por_genero_y_rango('rock', range(2018, 2023))

albumes_2018_2022 = tastyinfo_2018_2022['albumes']  # separar una lista de álbumes
canciones_2018_2022 = tastyinfo_2018_2022['canciones']  # separar una lista de canciones

# Aquí lo convertimos todo a Data Frame 

df_2018_2022_album = pd.DataFrame(albumes_2018_2022)
df_2018_2022_tracks = pd.DataFrame(canciones_2018_2022)

# PASO DE EXPORTACIÓN (de DataFrame a .csv):

#                         ---<-----<-----@ archivo para albumes a .CSV @----->---->---

df_2018_2022_album.to_csv('rock_2018_2022_albumes.csv', index=False, encoding='utf-8')

#                       --<-----<-----@ archivo para canciones a .CSV @----->---->---

df_2018_2022_tracks.to_csv('rock_2018_2022_canciones.csv', index=False, encoding='utf-8')

